In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

In [6]:
train_data = datasets.FashionMNIST(
    root = "data",
    train = True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root = "data",
    train = False,
    download=True,
    transform=ToTensor()
)

batch_size = 64
train_loader = DataLoader(train_data, batch_size=batch_size)
test_loader = DataLoader(test_data, batch_size=batch_size)

N_train, D1, D2 = train_data.data.shape
Nc = 10

In [8]:
class Model(nn.Module):
    def __init__(self, D1, D2):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 5, (2,2)),
            nn.ReLU(), 
            nn.Flatten(),
            nn.Linear(5*D1*D2, 10)
        )
    def forward(self, x):
        return self.cnn(x)

In [45]:
class Model(nn.Module):
    def __init__(self, D1, D2):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Flatten(),
            nn.Linear(32 * (D1 // 2) * (D2 // 2), 128),  # 입력 크기 수정
            nn.ReLU(),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        return self.cnn(x)

In [51]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
model = Model(D1, D2).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
loss_f = torch.nn.CrossEntropyLoss()
n_epoch = 2

for epoch in range(n_epoch):
    print(f"Epoch: {epoch}")
    for X, y in train_loader:
        X = X.to(device)
        y = y.to(device)
        pred = model(X)
        loss = loss_f(pred, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    train_acc = (pred.argmax(dim=1) == y).type(torch.float).sum().item() / len(y)
    train_loss = loss.item()
    print(f"Train: loss={train_loss:>.2e}, accuracy={train_acc:>.2f}")    

    with torch.no_grad():
        sz = len(test_loader.dataset)
        n_batch = len(test_loader)
        test_acc = 0
        test_loss = 0
        for X, y in test_loader:
            X = X.to(device)
            y = y.to(device)
            pred = model(X)
            test_acc += (pred.argmax(dim=1)==y).type(torch.float).sum().item()
            loss = loss_f(pred, y)
            test_loss += loss.item()
        test_acc /= sz
        test_loss /= n_batch
        print(f"Test") 

cuda
0: loss=1.43e+00, accuracy=0.66
1: loss=9.90e-01, accuracy=0.69
